In [ ]:
# Import library yang kita butuhkan
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Inisialisasi DataFrame kosong untuk menyimpan hasil scraping
df = pd.DataFrame()

# prepare list variables
productNames = []
productPrices = []
sellers = []
cities = []
soldQuantities = []
ratings = []

# Buat list kosong untuk menampung data produk
driver = webdriver.Chrome()

# URL target pencarian produk "seblak" di Tokopedia pada halaman pertama
url = "https://www.tokopedia.com/search?navsource=&ob=5&search_id=2025072106515111B9842E0A271F10E3H5&srp_component_id=04.06.00.00&srp_page_id=&srp_page_title=&st=&q=seblak"

# Memerintahkan browser untuk membuka halaman web
driver.get(url)

# Menambahkan delay 10 detik agar hasil screening lebih akurat
time.sleep(10)

# Ambil semua konten html dari halaman
html = driver.page_source
page = BeautifulSoup(html, "html.parser")

# Iterasi setiap produk
for product in page.find_all('div', {"class": "css-jza1fo"}):
    # Mencari setiap elemen produk
    name = product.find('div', {"class": "SzILjt4fxHUFNVT48ZPhHA=="})
    price = product.find('div', {"class": "urMOIDHH7I0Iy1Dv2oFaNw=="})
    seller = product.find('span', {"class": "si3CNdiG8AR0EaXvf6bFbQ== gxi+fsEljOjqhjSKqjE+sw== flip"})
    city = product.find('span', {"class": "gxi+fsEljOjqhjSKqjE+sw== flip"})
    sold = product.find('span', {"class": "u6SfjDD2WiBlNW7zHmzRhQ=="})
    rating = product.find('span', {"class": "_2NfJxPu4JC-55aCJ8bEsyw=="})
    # Menganmbil setiap elemen
    if name != None:
        productNames.append(name.get_text().strip())
    else:
        productNames.append(None)

    if price != None:
        productPrices.append(price.get_text().strip())
    else:
        productPrices.append(None)

    if seller != None:
        sellers.append(seller.get_text().strip())
    else:
        sellers.append(None)

    if city != None:
        cities.append(city.get_text().strip())
    else:
        cities.append(None)

    if sold != None:
        soldQuantities.append(sold.get_text().strip())
    else:
        soldQuantities.append(None)

    if rating != None:
        ratings.append(rating.get_text().strip())
    else:
        ratings.append(None)
    

# Memasukkan data ke dalam Data Frame
df['Nama Produk'] = productNames
df['Harga Produk'] = productPrices 
df['Penjual'] = sellers 
df['Kota Toko'] = cities 
df['Banyaknya Terjual'] = soldQuantities 
df['Rating Produk'] = ratings 

# Untuk menutup sesi browser
driver.quit()

# # Untuk menampilkan DataFrame hasil scraping
display(df)

In [ ]:
# Untuk menyimpan data ke file csv dengan nama "data_Penjualan_Seblak"
df.to_csv('data_Penjualan_Seblak.csv')

In [1]:
# Untuk import file csv 'data_Penjualan_Seblak.csv'
import pandas as pd
df_ori = pd.read_csv('data_Penjualan_Seblak.csv', index_col = 0)

In [2]:
# Untuk menampilkan 5(default) data teratas
df_ori.head()

,Nama Produk,Harga Produk,Penjual,Kota Toko,Banyaknya Terjual,Rating Produk
0,Seblak Mix Sagala Aya Ratubasreng 500 Gram Ped...,Rp20.000,Ratubasreng91,Kab. Bandung,100rb+ terjual,4.8
1,kylafood seblak varian tulang rawan,Rp21.165,Kylafood,Bandung,10rb+ terjual,4.9
2,kerupuk jengkol pedas 500gr/ seblak bantat,Rp20.999,ratihfood14,Kab. Bandung,10rb+ terjual,4.9
3,Seblak mix/ seblak campur/ seblak pedas daun j...,Rp19.800,diseblakanid,Kab. Bandung,10rb+ terjual,4.8
4,Kylafood Seblak Karuhun - Makanan Instan Pedas,Rp13.500,Kylafood,Bandung,6rb+ terjual,4.8


In [3]:
df_ori.info()
# Semua kolom masih bertipe object (string) karena hasil scraping berupa teks.
# Perlu dilakukan pembersihan dan konversi tipe data
# Harga produk diubah ke int atau float
# Banyaknya terjual diubah ke int atau float


<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Nama Produk        23 non-null     object 
 1   Harga Produk       23 non-null     object 
 2   Penjual            23 non-null     object 
 3   Kota Toko          23 non-null     object 
 4   Banyaknya Terjual  23 non-null     object 
 5   Rating Produk      23 non-null     float64
dtypes: float64(1), object(5)
memory usage: 1.3+ KB


In [4]:
# Menghilangkan "Rp" dan ".", misal Rp12.999 menjadi 12999
# Mengakses kolom “Harga Produk” dari DataFrame df.
# Menghapus string "Rp" dari setiap nilai di kolom
# Menghapus titik (.) yang merupakan pemisah ribuan dalam format harga.
# Mengubah hasil akhirnya dari string menjadi tipe integer
df_ori['Harga Produk'] = df_ori['Harga Produk'].str.replace('Rp', '').str.replace('.', '').astype(int)

In [5]:
# Mengubah kolom 'Banyaknya Terjual' yang sebelumnya seperti ini "Terjual 750+", "Terjual 1 rb" menjadi "750" dan "1000"
# Membuat fungsi dengan nama convert_sold_quantity yang menerima satu argumen quantity (isi dari satu baris kolom 'Banyaknya Terjual').
# Menghapus kata "terjual" (karena tidak diperlukan).
# .strip() digunakan untuk menghapus white space.
# Mengecek apakah string mengandung "rb" (singkatan dari ribu).
# Hapus "rb" dan simbol "+", lalu ubah sisanya menjadi angka dan kalikan 1000.
# Jika ada angka ubah jadi integer.
# Jika tidak ada angka (kosong) kembalikan 0.
# .apply(convert_sold_quantity) digunakan untuk menerapkan fungsi ke setiap baris kolom "Banyaknya Terjual"
def convert_sold_quantity(quantity):
    quantity = quantity.replace('terjual', '').strip()
    if 'rb' in quantity:
        return int(quantity.replace('rb', '').replace('+', '').strip()) * 1000
    else:
        numeric_part = ''.join(filter(str.isdigit, quantity))
        return int(numeric_part) if numeric_part else 0

# Apply the function and ensure the type is integer

df_ori['Banyaknya Terjual'] = df_ori['Banyaknya Terjual'].apply(convert_sold_quantity).astype(int)

In [ ]:
# Untuk melihat tabel yang sudah dibersihkan
df_ori

,Nama Produk,Harga Produk,Penjual,Kota Toko,Banyaknya Terjual,Rating Produk
0,Seblak Mix Sagala Aya Ratubasreng 500 Gram Ped...,20000,Ratubasreng91,Kab. Bandung,100000,4.8
1,kylafood seblak varian tulang rawan,21165,Kylafood,Bandung,10000,4.9
2,kerupuk jengkol pedas 500gr/ seblak bantat,20999,ratihfood14,Kab. Bandung,10000,4.9
3,Seblak mix/ seblak campur/ seblak pedas daun j...,19800,diseblakanid,Kab. Bandung,10000,4.8
4,Kylafood Seblak Karuhun - Makanan Instan Pedas,13500,Kylafood,Bandung,6000,4.8
5,SEBLAK CAMPUR/MIX ANEKA KERUPUK SEBLAK JENGKOL...,20250,KANGSEBLAK.ID,Kab. Bandung,10000,4.8
6,SEBLAK VIRAL/MIX SEBLAK CAMPURAN/MIX VIRAL REN...,11000,grosir cemilan pedas,Kab. Bandung,50000,4.6
7,Kylafood Paket Seblak Cup instan isi 2 PCS Pedas,28730,Kylafood,Bandung,10000,4.7
8,(5) Lima BUNGKUS SEBLAK INSTAN PAKET HEMAT BAN...,62880,TOKO BANGSATRIA,Kab.Ciamis,10000,4.9
9,SEBLAK CAMPUR/KERUPUK MIX PEDAS DAUN JERUK 1KG,41999,Rajalada_NEW,Kab. Bandung,4000,4.8


In [10]:
# Untuk mengecek apakah kolom "Harga Produk" dan "Banyaknya Terjual" sudah menjadi integer
df_ori.dtypes

Nama Produk           object
Harga Produk           int64
Penjual               object
Kota Toko             object
Banyaknya Terjual      int64
Rating Produk        float64
dtype: object